In [1]:
import numpy as np
import pandas as pd
import os

In [9]:
def read_gt(folder_path,file_name):
    os.chdir(folder_path)
    data = open(file_name,'r').read()
    return data

In [3]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [12]:
def convert_to_df(data):
    data_split1 = data.split("\n")
    num_data_split = []
    for row in data_split1:
        row_split = row.split(",")
        num_row_split = []
        for ele in row_split:
            if isfloat(ele):
                num_row_split.append(float(ele))
            else:
                num_row_split.append(np.NAN)
        num_data_split.append(num_row_split)
    data_df = pd.DataFrame(num_data_split)
    return data_df

In [13]:
def convert_txt_to_df(folder_path,file_name):
    data = read_gt(folder_path, file_name)
    data_df = convert_to_df(data)
    return data_df

In [14]:
carchase = convert_txt_to_df("D:\AeroMIT\dataset_rnn_yolo\TrackingDataset\Kalal\CarChase","gt.txt")
carchase

,0,1,2,3
0,145.00,84.000,199.00,121.00
1,147.57,84.742,201.41,121.64
2,152.91,85.897,206.98,122.94
3,155.44,86.041,209.45,123.04
4,158.24,86.171,212.47,123.33
5,163.72,86.726,218.00,123.92
6,166.26,87.005,220.37,124.08
7,168.84,87.410,222.89,124.45
8,171.98,87.983,225.89,124.93
9,173.94,88.158,227.76,125.03
